# Добавление необходимой библиотеки

In [1]:
import numpy as np

# Добавление фукнции для красивой печати матриц

In [2]:
def printA(a):
    for row in a:
        for col in row:
            print("{:8.1f}".format(col), end=" ")
        print("")

# Создание матриц для игры

Условие задачи "Семейный спор"

In [3]:
table = np.array([
    [
        [4, 0],
        [0, 1]
    ],
    [
        [1, 0],
        [0, 4]
    ],
])
print('Матрица A')
printA(table[0])
print('Матрица B')
printA(table[1])

Матрица A
     4.0      0.0 
     0.0      1.0 
Матрица B
     1.0      0.0 
     0.0      4.0 


Условие задачи "Дилемма заключенного"

In [4]:
table = np.array([
    [
        [-1, -10],
        [0, -8]
    ],
    [
        [-1, 0],
        [-10, -8]
    ],
])
print('Матрица A')
printA(table[0])
print('Матрица B')
printA(table[1])

Матрица A
    -1.0    -10.0 
     0.0     -8.0 
Матрица B
    -1.0      0.0 
   -10.0     -8.0 


Мое условие

In [5]:
table = np.array([
    [
        [4, 0, 2],
        [7, 2, 4],
        [1, 5, 3]
    ],
    [
        [4, 1, 3],
        [3, 4, 5],
        [2, 6, 4]
    ],
])
print('Матрица A')
printA(table[0])
print('Матрица B')
printA(table[1])

Матрица A
     4.0      0.0      2.0 
     7.0      2.0      4.0 
     1.0      5.0      3.0 
Матрица B
     4.0      1.0      3.0 
     3.0      4.0      5.0 
     2.0      6.0      4.0 


# Равновесие в доминирующих стратегиях

In [6]:
def find_dominate_line(matrix):  # Функция для нахождения доминирующей строки
    dominated = set()
    been_dominating = set()

    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                continue
            if all(map(lambda x: x[0] < x[1], zip(matrix[i], matrix[j]))):
                dominated.add(j)
                been_dominating.add(i)
    result = list(dominated.difference(been_dominating))
    return None if len(result) == 0 else result[0]

In [7]:
dominants = [find_dominate_line(table[i]) for i in range(len(table))]
print(dominants)

is_equilibrium = len(set(dominants)) == len([i for i in dominants if not i is None])
if is_equilibrium:
    print('Равновесие соблюдено')
else:
    print('Равновесие не соблюдено')

[1, None]
Равновесие не соблюдено


# Равновесие по Нэшу

In [8]:
from collections import Counter

A = [(i, j) for i, j in enumerate(np.argmax(table[0], axis=0))] # поиск точек с максимальным выигрышем для первого игрока в матрице A (ищу по строкам)
B = [(j, i) for i, j in enumerate(np.argmax(table[1], axis=1))] # поиск точек с максимальным выигрышем для первого игрока в матрице B (ищу по стобцам)
print(f'Точки для первого игрока: {A}')
print(f'Точки для второго игрока: {B}')
nesh = list((Counter(A) & Counter(B)).elements()) # Находим точки, принадлежащие обоим массивам

print(f'Точки равновесия по Нэшу: {nesh}')

Точки для первого игрока: [(0, 1), (1, 2), (2, 1)]
Точки для второго игрока: [(0, 0), (2, 1), (1, 2)]
Точки равновесия по Нэшу: [(1, 2), (2, 1)]


### Провожу преобразование матриц A и B к иному виду, более удобному для поиска равновесия по Парето

In [9]:
converted_table = [[(None,) for j in range(len(table[0][i]))] for i in range(len(table[0]))]
for i in range(len(converted_table)):
    for j in range(len(converted_table[i])):
        converted_table[i][j] = tuple([table[k][i][j] for k in range(len(table))])
converted_table

[[(4, 4), (0, 1), (2, 3)], [(7, 3), (2, 4), (4, 5)], [(1, 2), (5, 6), (3, 4)]]

# Равновесие по Парето

In [10]:
paretto_points = []
for i1 in range(len(converted_table)):
    for j1 in range(len(converted_table)): # перебор всех точек
        point1 = converted_table[i1][j1]
        dominating = []
        for i2 in range(len(converted_table)): # перебор всех точек для сравнения с point1
            for j2 in range(len(converted_table)):
                point2 = converted_table[i2][j2]
                # Если это не та же самая точка, одно значение которой не меньше, а другое строго больше, то считаем, что point2 "доминирует" над point1
                if not (i1 == i2 and j1 == j2) and (point2[0] >= point1[0] and point2[1] > point1[1]) or (point2[0] > point1[0] and point2[1] >= point1[1]):
                    dominating.append(point2)
        if len(dominating) == 0: # если точка не была "задоминирована", то это точка равновесия по Парето
            paretto_points.append(point1)

print(f'Точки равновесия по Парето: {paretto_points}')

Точки равновесия по Парето: [(7, 3), (5, 6)]


# Равновесие по Штакельбергу
Согласно теореме 8:
__Если игра двух лиц имеет по крайней мере две оптимальные по Парето и одновременно равновесные по Нэшу ситуации с различными векторами выигрышей,
то в игре имеет место борьба за лидерство.__

In [11]:
is_struggle_leadership = len(paretto_points) >= 2 and len(set(nesh)) != 1
if is_struggle_leadership:
    print('Имеет место борьба за лидерство')
else:
    print('Борьбы за лидерство нет')

H = []
for i in range(len(table)):
    leader_index = np.unravel_index(np.argmax(table[i]), table[i].shape)[i]
    sled_index = np.argmax(table[1-i], axis=1-i)[leader_index]
    H.append(tuple(table[i][leader_index][sled_index] for i in range(len(table))))

print(f'Точки равновесия по Штакельбергу: {H}')

Имеет место борьба за лидерство
Точки равновесия по Штакельбергу: [(4, 5), (4, 5)]
